In [2]:
import os
join = os.path.join
import numpy as np
from glob import glob
import torch
from segment_anything.build_sam3D import sam_model_registry3D
from segment_anything.utils.transforms3D import ResizeLongestSide3D
from segment_anything import sam_model_registry
from tqdm import tqdm
import argparse
import SimpleITK as sitk
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchio as tio
import numpy as np
from collections import OrderedDict, defaultdict
import json
import pickle
from utils.click_method import get_next_click3D_torch_ritm, get_next_click3D_torch_2
import utils.data_loader 
import einops as E
import torch
Dataset_Union_ALL_Val = utils.data_loader.Dataset_Union_ALL_Val


In [3]:
args = argparse.Namespace(
    test_data_path = '/home/t722s/Desktop/Datasets/BratsTestData/',
    checkpoint_path=  '',
    results_save_path='/home/t722s/Desktop/Sam-Med3DTest/evalBrats/',
    model='SAM-Med3D',
    seed=2023,
    dim = 3,
    crop_size = 128,
    data_type = 'Ts',
    split_num = 1,
    split_idx = 0,
)

points_path = os.path.join(args.results_save_path, 'prompts.pkl')


In [4]:
def compute_dice(mask_gt, mask_pred):
    """Compute soerensen-dice coefficient.
    Returns:
    the dice coeffcient as float. If both masks are empty, the result is NaN
    """
    volume_sum = mask_gt.sum() + mask_pred.sum()
    if volume_sum == 0:
        return np.NaN
    volume_intersect = (mask_gt & mask_pred).sum()
    return 2*volume_intersect / volume_sum

In [5]:
seg_old_path = '/home/t722s/Desktop/Sam-Med3DTest/segmentation_maps/BraTS2021_01646_pred_old0.nii.gz'
seg_new_path = '/home/t722s/Desktop/Sam-Med3DTest/segmentation_maps/BraTS2021_01646_pred_trans0.nii.gz'
label_path = '/home/t722s/Desktop/Datasets/BratsTestData/labelsTs/BraTS2021_01646.nii.gz'
image_path = '/home/t722s/Desktop/Datasets/BratsTestData/imagesTs/BraTS2021_01646.nii.gz'

seg_old = sitk.GetArrayFromImage(sitk.ReadImage(seg_old_path))
seg_new = sitk.GetArrayFromImage(sitk.ReadImage(seg_new_path))
label = sitk.GetArrayFromImage(sitk.ReadImage(label_path))

In [53]:
dim = 3
label = 1

infer_transform = [
        tio.ToCanonical(),
        tio.CropOrPad(mask_name='points_mask', target_shape=(args.crop_size,args.crop_size,args.crop_size)), # Will center the cropping/padding at the center of the bounding box of the clicks supplied.
    ]

transform = tio.Compose(infer_transform)

with open(points_path, 'rb') as f:
    points_dict = pickle.load(f)

tio_image = tio.ScalarImage.from_sitk(sitk.ReadImage(image_path))


#### UNDER CONSTRUCTION
# Load in points for this image and change to a mask to be usable by torchio
points = points_dict[os.path.basename(image_path)][label][str(dim) + 'D']
points_mask = np.zeros(shape = (tio_image.shape[3], tio_image.shape[2], tio_image.shape[1])) # Skip the color channel for now; reintroduce in label_map definition. Reverse order since sitk uses WHD while numpy uses DHW
points_mask[*points.T] = 1
points_mask = E.rearrange(points_mask, pattern = 'x y z -> z y x') # Rearrange back to sitk WHD

points_mask = tio.LabelMap(tensor = torch.from_numpy(points_mask).float().unsqueeze(0), affine = tio_image.affine)

subject = tio.Subject(
    image = tio_image,
    points_mask = points_mask,
    seg = tio.LabelMap(tensor = torch.from_numpy(seg_new.transpose(2,1,0)).float().unsqueeze(0), affine = tio_image.affine),
    label = tio.ScalarImage.from_sitk(sitk.ReadImage(label_path)),
)

if '/ct_' in image_path:
    subject = tio.Clamp(-1000,1000)(subject)

subject = transform(subject)


In [54]:
subject['seg'].data.shape

torch.Size([1, 128, 128, 128])

In [49]:
seg_new = seg_new.transpose(2,1,0)
compute_dice(label, seg_new)

0.762314240369355

In [57]:
t = compute_dice(subject['label'].data[0], subject['seg'].data[0].bool())
t.item()

0.7623142600059509

In [50]:
t = compute_dice(subject['label'].data[0], seg_old)
t.item()

0.7623142600059509